In [1]:
pip install cirq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [2]:
import cirq

# Pick a qubit.
qubit = cirq.GridQubit(0, 0)

# Create a circuit
circuit = cirq.Circuit(
    cirq.X(qubit)**0.5,  # Square root of NOT.
    cirq.measure(qubit, key='m')  # Measurement.
)
print("Circuit:")
print(circuit)

# Simulate the circuit several times.
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=20)
print("Results:")
print(result)

Circuit:
(0, 0): ───X^0.5───M('m')───
Results:
m=01110010100100111000


In [3]:
import cirq
import numpy as np
import sympy as sp

# Define two qubits to represent two "words"
q1, q2 = cirq.LineQubit.range(2)

# Define a parameter for a variational layer (for example, a rotation angle)
theta = sp.Symbol('theta')

# Build a simple quantum circuit representing a minimal NLP pipeline:
# 1. Prepare word representations.
# 2. Apply a parameterized gate (variational layer) to mix information.
# 3. Measure an observable that might indicate similarity or another property.

circuit = cirq.Circuit()

# Prepare word representations:
# Let's assume |w1> = H|0> and |w2> = XH|0>
circuit.append([cirq.H(q1), cirq.H(q2)])         # initial Hadamards
circuit.append(cirq.X(q2))                         # apply X to second qubit

# Apply a variational layer: e.g., Rx rotation on both qubits with parameter theta.
circuit.append([cirq.rx(theta)(q1), cirq.rx(theta)(q2)])

# Optionally, add an entangling gate (e.g., CNOT) to mix the representations.
circuit.append(cirq.CNOT(q1, q2))

# Measure in the computational basis.
circuit.append([cirq.measure(q1, key='m1'), cirq.measure(q2, key='m2')])

print("Circuit:")
print(circuit)

# Now, suppose we want to run this circuit using a parameterized simulation.
# Create a simulator.
simulator = cirq.Simulator()

# Create a parameter resolver for a specific value of theta.
param_resolver = cirq.ParamResolver({theta: np.pi / 4})

# Run the circuit (with a few repetitions).
result = simulator.run(circuit, param_resolver=param_resolver, repetitions=10)
print("\nMeasurement results:")
print(result)


Circuit:
0: ───H───Rx(theta)───────────────@───M('m1')───
                                  │
1: ───H───X───────────Rx(theta)───X───M('m2')───

Measurement results:
m1=0011000001
m2=1101111111


In [4]:
import cirq
import numpy as np
import sympy as sp

# A very basic function to convert text to a quantum parameter.
# (Here we sum the ASCII codes and scale it to get an angle between 0 and π.)
def text_to_param(text):
    total = sum(ord(c) for c in text)
    # Scale total to an angle: mod 10 gives a number between 0 and 9.
    theta = (total % 10) / 10 * np.pi
    return theta

# A simple retrieval mapping: measurement outcome -> response.
retrieval_map = {
    0: "Response: The quantum answer is A.",
    1: "Response: The quantum answer is B.",
    2: "Response: The quantum answer is C.",
    3: "Response: The quantum answer is D."
}

# Build a quantum circuit that uses a parameterized rotation.
def build_circuit(theta):
    # Create a single qubit.
    q = cirq.LineQubit(0)
    circuit = cirq.Circuit()
    # Apply an Rx rotation with the given angle.
    circuit.append(cirq.rx(theta)(q))
    # Measure the qubit.
    circuit.append(cirq.measure(q, key='m'))
    return circuit

# Example: Convert a text query into a quantum circuit and simulate retrieval.
text_input = "hello quantum"
theta = text_to_param(text_input)
print("Text input:", text_input)
print("Encoded rotation angle (theta): {:.3f} rad".format(theta))

# Build the circuit with our parameter.
circuit = build_circuit(theta)
print("\nQuantum circuit:")
print(circuit)

# Simulate the circuit.
simulator = cirq.Simulator()
# Run with a number of repetitions to get a measurement histogram.
result = simulator.run(circuit, repetitions=100)
measurement = result.histogram(key='m')
print("\nMeasurement histogram:", dict(measurement))

# For retrieval, choose the most common measurement outcome.
most_common = max(measurement.items(), key=lambda x: x[1])[0]
# Map the outcome to a response using modulo if needed.
retrieval_index = most_common % len(retrieval_map)
response = retrieval_map[retrieval_index]
print("\nRetrieved response:")
print(response)


Text input: hello quantum
Encoded rotation angle (theta): 0.942 rad

Quantum circuit:
0: ───Rx(0.3π)───M('m')───

Measurement histogram: {1: 17, 0: 83}

Retrieved response:
Response: The quantum answer is A.


In [5]:
import cirq
import numpy as np

# 1. Define a function that converts a text string into a pair of rotation angles.
def text_to_angles(text):
    # Compute two aggregated values from the text.
    total1 = sum(ord(c) for c in text)
    total2 = sum(ord(c)**2 for c in text)
    # Scale these totals to yield angles between 0 and π.
    theta1 = (total1 % 100) / 100 * np.pi
    theta2 = (total2 % 100) / 100 * np.pi
    return theta1, theta2

# 2. Build a quantum circuit for a given pair of angles.
def build_circuit_for_angles(theta1, theta2):
    # We'll use 2 qubits to encode the two angles.
    q0, q1 = cirq.LineQubit.range(2)
    circuit = cirq.Circuit()
    # Apply Rx rotations on each qubit.
    circuit.append(cirq.rx(theta1)(q0))
    circuit.append(cirq.rx(theta2)(q1))
    return circuit, [q0, q1]

# 3. Compute expectation values for Z on each qubit given a circuit.
def compute_expectations(circuit, qubits):
    simulator = cirq.Simulator()
    # Simulate the circuit; we specify qubit_order to ensure consistent ordering.
    result = simulator.simulate(circuit, qubit_order=qubits)
    state = result.final_state_vector
    num_qubits = len(qubits)
    exp_vals = []
    # Cirq uses big-endian ordering: the first qubit is the most significant bit.
    for i in range(num_qubits):
        exp_val = 0.0
        for index, amplitude in enumerate(state):
            prob = np.abs(amplitude)**2
            # Extract the bit for qubit i (with i=0 as most significant).
            bit = (index >> (num_qubits - 1 - i)) & 1
            # Contribution: +1 if the qubit is in |0>, -1 if in |1>.
            exp_val += (1 if bit == 0 else -1) * prob
        exp_vals.append(exp_val)
    return np.array(exp_vals)

# 4. Create a corpus of questions and answers.
corpus = {
    "What is quantum computing?": "Quantum computing harnesses quantum mechanics to process information in ways that classical computers cannot.",
    "What is a qubit?": "A qubit is the quantum version of a classical bit and can exist in superposition of 0 and 1.",
    "How does quantum entanglement work?": "Quantum entanglement is a phenomenon where two or more particles become correlated in ways that classical physics cannot explain.",
    "What is superposition?": "Superposition allows a quantum system to be in multiple states at once until a measurement collapses it."
}

# Precompute the expectation vectors for each corpus question.
corpus_expectations = {}
for question, answer in corpus.items():
    angles = text_to_angles(question)
    circuit, qubits = build_circuit_for_angles(*angles)
    exp_vector = compute_expectations(circuit, qubits)
    corpus_expectations[question] = (exp_vector, answer)
    print(f"Corpus Q: '{question}' -> Angles: {angles}, Expectation: {exp_vector}")

# 5. Process a query.
query = "Tell me about qubits."
query_angles = text_to_angles(query)
query_circuit, query_qubits = build_circuit_for_angles(*query_angles)
query_exp = compute_expectations(query_circuit, query_qubits)
print("\nQuery:", query)
print("Query angles:", query_angles)
print("Query expectation vector:", query_exp)

# 6. Compare the query expectation vector to each corpus vector using Euclidean distance.
def euclidean_distance(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

best_match = None
best_distance = None
for q_text, (exp_vec, answer) in corpus_expectations.items():
    dist = euclidean_distance(query_exp, exp_vec)
    print(f"Distance from query to '{q_text}': {dist:.3f}")
    if best_distance is None or dist < best_distance:
        best_distance = dist
        best_match = (q_text, answer)

print("\nBest match:")
print("Question:", best_match[0])
print("Answer:", best_match[1])


Corpus Q: 'What is quantum computing?' -> Angles: (1.382300767579509, 1.8849555921538759), Expectation: [ 0.18738127 -0.30901705]
Corpus Q: 'What is a qubit?' -> Angles: (0.91106186954104, 2.921681167838508), Expectation: [ 0.61290717 -0.97591687]
Corpus Q: 'How does quantum entanglement work?' -> Angles: (1.0053096491487339, 2.3876104167282426), Expectation: [ 0.53582676 -0.72896861]
Corpus Q: 'What is superposition?' -> Angles: (2.9845130209103035, 2.796017461694916), Expectation: [-0.98768828 -0.94088071]

Query: Tell me about qubits.
Query angles: (1.7592918860102844, 0.06283185307179587)
Query expectation vector: [-0.18738128  0.99802669]
Distance from query to 'What is quantum computing?': 1.360
Distance from query to 'What is a qubit?': 2.130
Distance from query to 'How does quantum entanglement work?': 1.872
Distance from query to 'What is superposition?': 2.098

Best match:
Question: What is quantum computing?
Answer: Quantum computing harnesses quantum mechanics to process in

In [6]:
import cirq
import numpy as np

# --------------------------------------------------------------------
# 1. Text-to-Angles Conversion
# A simple function that maps a text string into a pair of rotation angles.
# Here we use two aggregated values (sum of ASCII codes and sum of their squares)
# and scale them to get angles between 0 and π.
def text_to_angles(text):
    total1 = sum(ord(c) for c in text)
    total2 = sum(ord(c)**2 for c in text)
    theta1 = (total1 % 100) / 100 * np.pi
    theta2 = (total2 % 100) / 100 * np.pi
    return theta1, theta2

# --------------------------------------------------------------------
# 2. Circuit Construction from Angles
# Given two angles, we build a quantum circuit on 2 qubits.
def build_circuit_for_angles(theta1, theta2):
    # Use two line qubits to encode our two angles.
    q0, q1 = cirq.LineQubit.range(2)
    circuit = cirq.Circuit()
    # Apply Rx rotations to each qubit
    circuit.append(cirq.rx(theta1)(q0))
    circuit.append(cirq.rx(theta2)(q1))
    # For clarity, we print the circuit diagram.
    print("Constructed Circuit:")
    print(circuit)
    return circuit, [q0, q1]

# --------------------------------------------------------------------
# 3. Compute Expectation Values
# Simulate the circuit and compute the expectation value of Z on each qubit.
def compute_expectations(circuit, qubits):
    simulator = cirq.Simulator()
    # Use a full wavefunction simulation to get the final state vector.
    result = simulator.simulate(circuit, qubit_order=qubits)
    state = result.final_state_vector
    num_qubits = len(qubits)
    exp_vals = []
    # For each qubit, compute expectation value of Z.
    # In the computational basis, |0> yields +1 and |1> yields -1.
    for i in range(num_qubits):
        exp_val = 0.0
        for index, amplitude in enumerate(state):
            prob = np.abs(amplitude)**2
            # Extract the bit for qubit i (0 is most significant).
            bit = (index >> (num_qubits - 1 - i)) & 1
            exp_val += (1 if bit == 0 else -1) * prob
        exp_vals.append(exp_val)
    exp_vector = np.array(exp_vals)
    print("Expectation vector for circuit:", exp_vector)
    return exp_vector

# --------------------------------------------------------------------
# 4. Create a Corpus of Q/A Pairs
corpus = {
    "What is quantum computing?": "Quantum computing harnesses quantum mechanics to process information in ways that classical computers cannot.",
    "What is a qubit?": "A qubit is the quantum version of a classical bit that can exist in a superposition of 0 and 1.",
    "How does quantum entanglement work?": "Entanglement means two particles become correlated such that the state of one immediately influences the state of the other.",
    "What is superposition?": "Superposition allows a quantum system to exist in multiple states simultaneously until measured."
}

# Precompute expectation vectors for each corpus question.
print("=== Corpus Preprocessing ===")
corpus_expectations = {}
for question, answer in corpus.items():
    angles = text_to_angles(question)
    circuit, qubits = build_circuit_for_angles(*angles)
    exp_vector = compute_expectations(circuit, qubits)
    corpus_expectations[question] = (exp_vector, answer)
    print(f"\nCorpus Q: '{question}'")
    print(f"  Angles: {angles}")
    print(f"  Expectation: {exp_vector}")
    print("-" * 50)

# --------------------------------------------------------------------
# 5. Process a Query and Retrieve an Answer
query = "Tell me about qubits."
print("\n=== Processing Query ===")
query_angles = text_to_angles(query)
query_circuit, query_qubits = build_circuit_for_angles(*query_angles)
query_exp = compute_expectations(query_circuit, query_qubits)
print("\nQuery:", query)
print("Query angles:", query_angles)
print("Query expectation vector:", query_exp)

# --------------------------------------------------------------------
# 6. Compare Query to Corpus and Retrieve Best Answer
def euclidean_distance(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

best_match = None
best_distance = None
for q_text, (exp_vec, answer) in corpus_expectations.items():
    dist = euclidean_distance(query_exp, exp_vec)
    print(f"Distance from query to '{q_text}': {dist:.3f}")
    if best_distance is None or dist < best_distance:
        best_distance = dist
        best_match = (q_text, answer)

print("\n=== Retrieval Result ===")
print("Best matching question:")
print(best_match[0])
print("Associated answer:")
print(best_match[1])


=== Corpus Preprocessing ===
Constructed Circuit:
0: ───Rx(0.44π)───

1: ───Rx(0.6π)────
Expectation vector for circuit: [ 0.18738127 -0.30901705]

Corpus Q: 'What is quantum computing?'
  Angles: (1.382300767579509, 1.8849555921538759)
  Expectation: [ 0.18738127 -0.30901705]
--------------------------------------------------
Constructed Circuit:
0: ───Rx(0.29π)───

1: ───Rx(0.93π)───
Expectation vector for circuit: [ 0.61290717 -0.97591687]

Corpus Q: 'What is a qubit?'
  Angles: (0.91106186954104, 2.921681167838508)
  Expectation: [ 0.61290717 -0.97591687]
--------------------------------------------------
Constructed Circuit:
0: ───Rx(0.32π)───

1: ───Rx(0.76π)───
Expectation vector for circuit: [ 0.53582676 -0.72896861]

Corpus Q: 'How does quantum entanglement work?'
  Angles: (1.0053096491487339, 2.3876104167282426)
  Expectation: [ 0.53582676 -0.72896861]
--------------------------------------------------
Constructed Circuit:
0: ───Rx(0.95π)───

1: ───Rx(0.89π)───
Expectation v